In [3]:
import sllurp

In [1]:
# Minimal example; see inventory.py for more.
from sllurp import llrp
from twisted.internet import reactor
import logging
logging.getLogger().setLevel(logging.INFO)

def cb (tagReport):
    tags = tagReport.msgdict['RO_ACCESS_REPORT']['TagReportData']
    print 'tags:', tags

factory = llrp.LLRPClientFactory()
factory.addTagReportCallback(cb)
reactor.connectTCP('myreader', llrp.LLRP_PORT, factory)
reactor.run()

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named service_identity'.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.
No handlers could be found for logger "sllurp.llrp"


In [ ]:
llrp.LLRPClientFactory(tag_content_selector={
    'EnableROSpecID': False,
    'EnableSpecIndex': False,
    'EnableInventoryParameterSpecID': False,
    'EnableAntennaID': True,
    'EnableChannelIndex': False,
    'EnablePeakRSSI': True,
    'EnableFirstSeenTimestamp': False,
    'EnableLastSeenTimestamp': True,
    'EnableTagSeenCount': True,
    'EnableAccessSpecID': False,
}, ...)

In [15]:
#To see what inventory settings an Impinj reader is 
#currently using (i.e., to fetch the current ROSpec), ssh to the reader and
#show rfid llrp rospec 0

#The “nuclear option” for resetting a reader is
#reboot
cb?

In [9]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(__file__, '..', '..', '..')))

from argparse import ArgumentParser
from logging import getLogger, INFO, Formatter, StreamHandler, WARN
from sllurp.llrp import LLRP_PORT, LLRPClientFactory
import smokesignal
from tornado.escape import json_decode
from tornado.platform.twisted import TwistedIOLoop
from tornado.template import Loader
from tornado.web import RequestHandler, Application
from tornado.websocket import WebSocketClosedError, WebSocketHandler
from twisted.internet import reactor

'''
Sllurp/Tornado Example
This file contains an example showing how to use Sllurp with Tornado
to update a web page via websockets when rfid tags are seen.
'''

logger = getLogger('sllurp')


def setup_logging():
    logger.setLevel(INFO)
    logFormat = '%(asctime)s %(name)s: %(levelname)s: %(message)s'
    formatter = Formatter(logFormat)
    handler = StreamHandler()
    handler.setFormatter(formatter)
    logger.addHandler(handler)

    stream_handler_warn = StreamHandler()
    stream_handler_warn.setLevel(WARN)
    stream_handler_warn.setFormatter(formatter)

    access_log = getLogger("tornado.access")
    access_log.addHandler(stream_handler_warn)

    app_log = getLogger("tornado.application")
    app_log.addHandler(handler)

    gen_log = getLogger("tornado.general")
    gen_log.addHandler(handler)


class DefaultHandler(RequestHandler):
    def get(self):
        loader = Loader(os.path.dirname(__file__))
        template = loader.load("index.html")
        self.write(template.generate())


class WebSocketHandler(WebSocketHandler):
    def __init__(self, *args, **kwargs):
        super(WebSocketHandler, self).__init__(*args, **kwargs)
        self.connected_clients = 0

        @smokesignal.on('rfid')
        def _RFID_cb(payload):
            self.update_rfid(payload)

    def open(self):
        self.connected_clients += 1
        logger.debug("WebSocket opened")

    def on_message(self, message):
        try:
            data = json_decode(message)
            logger.info(data)
        except ValueError:
            logger.info('error loading json: {}'.format(message))

    def on_close(self):
        self.connected_clients -= 1
        logger.debug("WebSocket closed")

    def update_rfid(self, payload):
        if self.connected_clients > 0:
            try:
                self.write_message(payload)
                # logger.debug('websocket write: {}'.format(pformat(payload)))
            except WebSocketClosedError:
                logger.debug('attempting to send websocket message with no connected clients')


def tag_seen_callback(llrpMsg):
        """Function to run each time the reader reports seeing tags."""
        tags = llrpMsg.msgdict['RO_ACCESS_REPORT']['TagReportData']
        if tags:
            smokesignal.emit('rfid', {
                'tags': tags,
            })


def parse_args():
    parser = ArgumentParser(description='Simple RFID Reader Inventory')
    parser.add_argument('host', help='hostname or IP address of RFID reader',
                        nargs='*')
    parser.add_argument('-p', '--port', default=LLRP_PORT, type=int,
                        help='port to connect to (default {})'.format(LLRP_PORT))
    parser.add_argument('-n', '--report-every-n-tags', default=1, type=int,
                        dest='every_n', metavar='N', help='issue a TagReport every N tags')
    parser.add_argument('-a', '--antennas', default='1',
                        help='comma-separated list of antennas to enable')
    parser.add_argument('-X', '--tx-power', default=0, type=int,
                        dest='tx_power', help='Transmit power (default 0=max power)')
    parser.add_argument('-M', '--modulation', default='M8',
                        help='modulation (default M8)')
    parser.add_argument('-T', '--tari', default=0, type=int,
                        help='Tari value (default 0=auto)')
    return parser.parse_args()


def polite_shutdown(factory):
    return factory.politeShutdown()


if __name__ == '__main__':
    setup_logging()
    # Set up tornado to use reactor
    TwistedIOLoop().install()

    # Set up web server
    application = Application([(r"/", DefaultHandler),
                              (r"/ws", WebSocketHandler)])
    application.listen(8888)

    # Load Sllurp config
    args = parse_args()
    enabled_antennas = list(
        map(lambda x: int(x.strip()), args.antennas.split(',')))

    # Create Clients and set them to connect
    fac = LLRPClientFactory(report_every_n_tags=args.every_n,
                            antennas=enabled_antennas,
                            tx_power=args.tx_power,
                            modulation=args.modulation,
                            start_inventory=True,
                            tari=args.tari,
                            tag_content_selector={
                                'EnableROSpecID': True,
                                'EnableSpecIndex': True,
                                'EnableInventoryParameterSpecID': True,
                                'EnableAntennaID': True,
                                'EnableChannelIndex': True,
                                'EnablePeakRSSI': True,
                                'EnableFirstSeenTimestamp': True,
                                'EnableLastSeenTimestamp': True,
                                'EnableTagSeenCount': True,
                                'EnableAccessSpecID': True,
                            })
    fac.addTagReportCallback(tag_seen_callback)

    for host in args.host:
        reactor.connectTCP(host, args.port, fac, timeout=3)

    reactor.addSystemEventTrigger('before', 'shutdown', polite_shutdown, fac)

    # Start server & connect to readers
reactor.run()

NameError: name '__file__' is not defined